In [ ]:
!python -V

In [ ]:
# Função para ver quais são os parâmetros de cada operador 
def get_operator_info(operator):

    op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)
    print('Op name:', op_spi.getOperatorDescriptor().getName())
    print('Op alias:', op_spi.getOperatorDescriptor().getAlias()) 
    param_Desc = op_spi.getOperatorDescriptor().getParameterDescriptors()
    for param in param_Desc:
          print(param.getName(), "or", param.getAlias())

In [ ]:
# Bibliotecas
import os
import snappy
from snappy import GPF
from snappy import ProductIO
from snappy import HashMap
from snappy import jpy
from snappy import ProductUtils
import subprocess

In [ ]:
# Acessando os operadores Java
HashMap = jpy.get_type('java.util.HashMap')
parameters = HashMap()

# Função para ler arquivo
def read(file):
    return ProductIO.readProduct(file)

# Função para escrever arquivo em disco
def write(product, file):
    ProductIO.writeProduct(product, file, "GeoTIFF")

In [ ]:
# TOPSAR-Split
def split(product):
    parameters.put('subswath', 'IW1')
    parameters.put('selectedPolarisations', 'VV')
    parameters.put('firstBurstIndex', '4')
    parameters.put('lastBurstIndex', '5')
    return GPF.createProduct("TOPSAR-Split", parameters, product)

# Orbit files
def apply_orbit_file(product):   
    parameters.put("Orbit State Vectors", "Sentinel Precise (Auto Download)")
    parameters.put("Polynomial Degree", 3)    
    return GPF.createProduct("Apply-Orbit-File", parameters, product)

# Back Geocoding
def back_geocoding(product):    
    parameters.put("Digital Elevation Model", "SRTM 1Sec HGT (Auto Download)")
    parameters.put("DEM Resampling Method", "BICUBIC_INTERPOLATION")
    parameters.put("Resampling Type", "BISINC_5_POINT_INTERPOLATION")
    parameters.put("Mask out areas with no elevation", True)
    parameters.put("Output Deramp and Demod Phase", False)    
    return GPF.createProduct("Back-Geocoding", parameters, product)

In [ ]:
# Interferograma
def interferogram(product):  
    parameters.put("Subtract flat-earth phase", True)
    parameters.put("Degree of \"Flat Earth\" polynomial", 5)
    parameters.put("Number of \"Flat Earth\" estimation points", 501)
    parameters.put("Orbit interpolation degree", 3)
    parameters.put("Include coherence estimation", True)
    parameters.put("Square Pixel", False)
    parameters.put("Independent Window Sizes", False)
    parameters.put("Coherence Azimuth Window Size", 10)
    parameters.put("Coherence Range Window Size", 10)
    return GPF.createProduct("Interferogram", parameters, product)

# Deburst
def deburst(product):
    parameters.put("Polarisations", "VV")
    return GPF.createProduct("TOPSAR-Deburst", parameters, product)

# Multilooking
def multilook(product):
    parameters.put("Number of Range Looks", 4)
    parameters.put("Number of Azimuth Looks", 1)
    return GPF.createProduct("Multilook", parameters, product)

# Remover fase topográfica
def topophase_removal(product):
    parameters.put("Orbit Interpolation Degree", 3)
    parameters.put("Digital Elevation Model", "SRTM 1Sec HGT (Auto Download)")
    parameters.put("Tile Extension[%]", 100)
    parameters.put("Output topographic phase band", True)
    parameters.put("Output elevation band", False)
    return GPF.createProduct("TopoPhaseRemoval", parameters, product)

In [ ]:
# Filtro de Goldstein para a fase
def goldstein_filtering(product):
    parameters.put("Adaptive Filter Exponent in(0,1]:", 1.0)
    parameters.put("FFT Size", 64)
    parameters.put("Window Size", 3)
    parameters.put("Use coherence mask", False)
    parameters.put("Coherence Threshold in[0,1]:", 0.2)
    return GPF.createProduct("GoldsteinPhaseFiltering", parameters, product)

# Converter fase desdobrada em deslocamento
def displacement(product):
    return GPF.createProduct("PhaseToDisplacement", product)

# Correção geométrica
def terrain_correction(product):
    parameters.put('Digital Elevation Model', 'SRTM 1Sec HGT (Auto Download)')
    parameters.put("DEM Resampling Method", "BICUBIC_INTERPOLATION")
    parameters.put("Image Resampling Method", "BICUBIC_INTERPOLATION")
    parameters.put('pixelSpacingInMeter', 15.0) 
    parameters.put('sourceBands', 'Phase_ifg_VV_07Jun2021_25Jul2021')
    return GPF.createProduct("Terrain-Correction",parameters, product)

In [ ]:
# Fluxo de processos
def workflow(file_1, file_2):

    print('Reading files')
    product_1 = read(file_1)
    product_2 = read(file_2)

    print('TOPSAR-Split')
    SPLIT_1 = split(product_1)
    SPLIT_2 = split(product_2)

    print('Applying orbit files')
    product_orbitFile_1 = apply_orbit_file(SPLIT_1)
    product_orbitFile_2 = apply_orbit_file(SPLIT_2)

    print('Back geocoding')
    backgeocoding = back_geocoding([product_orbitFile_1, product_orbitFile_2])

    print('Interferogram generation')
    interferogram_formation = interferogram(backgeocoding)

    print('TOPSAR_deburst')
    TOPSAR_deburst = deburst(interferogram_formation)

    print('TopoPhase removal')
    TOPO_phase_removal = topophase_removal(TOPSAR_deburst)
    
    print('Multilooking')
    multi = multilook(TOPO_phase_removal)

    print('Goldstein filtering')
    gold = goldstein_filtering(multi)
    
    
    print('Terrain Correction')
    corr = terrain_correction(gold)

    print('Write')
    #write(gold, output)
        
    write(corr, output)

In [ ]:
# Data 1
master = os.path.join('D:/S1A_IW_SLC__1SDV_20210725T080234_20210725T080301_038931_0497F8_CE2A.zip')

# Data 2
slave = os.path.join('D:/S1A_IW_SLC__1SDV_20210607T080231_20210607T080258_038231_048301_9CEE.zip')

# Nome do arquivo final
output= 'Interf.tif'

workflow(master, slave)

In [ ]:
# Lendo fase desdobrada
unwrapp = read('Interf_snaphu.tif')

# Calculando deslocamento
dis = displacement(unwrapp)

write(dis, 'displacement')

In [4]:
!jt -t monokai